In [ ]:
!pip install pyservicebinding
!pip install psycopg2-binary


In [ ]:
import os
import pyservicebinding
from pyservicebinding import binding

if 'SERVICE_BINDING_ROOT' in os.environ:
    print('DB binding ok')
else:
    print('DB binding missing')

In [ ]:
sb = binding.ServiceBinding()
bindings_list = sb.bindings('postgresql', 'Red Hat DBaaS / Crunchy Bridge')

In [ ]:
import psycopg2
import psycopg2.errorcodes

In [ ]:
import psycopg2
import psycopg2.errorcodes
import time
import logging
import random
db_connection = psycopg2.connect(database=bindings_list[0].get('database'), \
    user=bindings_list[0].get('username'), \
    password=bindings_list[0].get('password'), \
    host=bindings_list[0].get('host'), \
    port=bindings_list[0].get('port'))
print(db_connection)

In [ ]:
import pandas as pd
cur = db_connection.cursor()
cur.execute("SELECT * FROM bluebook_small")
logging.debug("select_all(): status message: {}".format(cur.statusmessage))
df = pd.DataFrame(cur.fetchall(), columns = ['Price','Mileage','Make','Model','Trim','Type','Cylinder','Liter','Doors','Cruise','Sound','Leather'])
df.head()

In [ ]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile
from IPython.display import display, Latex, Markdown
#import plotly.express as px
#import plotly.graph_objs as go
plt.rcParams['font.sans-serif'] = "Comic Sans MS"
import warnings; warnings.simplefilter('ignore')
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder()
df['Make']= le.fit_transform(df['Make']) 
df['Model']= le.fit_transform(df['Model']) 
df['Trim']= le.fit_transform(df['Trim']) 
df['Type']= le.fit_transform(df['Type'])
df

In [ ]:
X = df.drop('Price', 1)
X = X.apply(pd.to_numeric)
y = df['Price']
y = y.apply(pd.to_numeric)

In [ ]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X[['Mileage', 'Make', 'Model', 'Trim', 'Type', 'Cylinder', 'Liter', 'Doors', 'Cruise', 'Sound', 'Leather']] = scale.fit_transform(X[['Mileage', 'Make', 'Model', 'Trim', 'Type', 'Cylinder', 'Liter', 'Doors', 'Cruise', 'Sound', 'Leather']].values)

In [ ]:
X

In [ ]:
# evaluate RFE for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

# create pipeline
rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=5)
model = DecisionTreeRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
feature_name = X.columns.tolist()
num_feats = len(feature_name)

In [ ]:
# create pipeline
from sklearn.model_selection import train_test_split
rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=5)
model = DecisionTreeRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# fit the model on all available data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
pipeline.fit(X_train, y_train)
yhat = pipeline.predict(X_test)
#print(yhat)
# summarize all features
for i in range(X_train.shape[1]):
    print('Column: %d, %s, Selected %s, Rank: %.3f' % (i, feature_name[i], rfe.support_[i], rfe.ranking_[i]))
print("PREDICTED VALUE\n", pd.DataFrame(data=yhat[:20]), "\nTEST VALUE\n", y_test[:20])

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
fig = go.Figure()
count = 20
fig.add_trace(go.Scatter(x = X_test[:count], y = y_test[:count], mode = 'markers', name = 'actual', marker=dict(size=10,color=1)))
fig.add_trace(go.Scatter(x = X_test[:count], y = yhat[:count], mode = 'markers', name = 'predictions', marker=dict(size=10,color=3)))
fig.update_layout( yaxis_title = 'Price')